In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder


#import models
# from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor
# from sklearn.svm import SVR

# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submit.csv')

In [4]:
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

In [5]:
to_drop = []

In [6]:
for col in train.columns:
    if col == 'target' or col == 'CTR_CATEGO_X' or col == 'id':
        continue
    else:
        if train[col].max() <= 1:
            to_drop.append(col)

In [7]:
for col in train.columns:
    if train[col].isna().sum() >= 16000:
        to_drop.append(col)

In [8]:
len(to_drop)

40

In [9]:
train.shape

(25053, 121)

In [10]:
test.shape

(10738, 120)

In [11]:
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)

In [12]:
train['id_freq'] = train['id'].map(train.groupby('id').size()/len(train))
test['id_freq'] = test['id'].map(test.groupby('id').size()/len(test))

In [13]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [14]:
indices = []

In [15]:
for i in range(train.shape[0]):
    if train['CTR_CATEGO_X'][i] == 'N':
        indices.append(i)

In [16]:
indices

[883, 2012, 2620, 4314, 11843, 16559, 17014, 18485, 19294, 19472, 23361]

In [17]:
train.drop(indices, axis=0, inplace=True)

In [18]:
encode = LabelEncoder()

In [19]:
train['CTR_CATEGO_X'] = encode.fit_transform(train['CTR_CATEGO_X'])
test['CTR_CATEGO_X'] = encode.fit_transform(test['CTR_CATEGO_X'])

In [21]:
train.isna().sum()

BCT_CODBUR         0
CTR_MATFIS         0
CTR_CATEGO_X       0
FJU_CODFJU         0
CTR_CESSAT         0
ACT_CODACT         0
CTR_OBLDIR         0
CTR_OBLACP         0
CTR_OBLRES         0
CTR_OBLFOP         0
CTR_OBLTFP         0
CTR_OBLDCO         0
CTR_OBLTVA         0
CTR_OFODEC        10
CTR_OFODEP        10
CTR_OFODET        10
CTR_OBLAUT        10
CTR_OBLASS        10
CTR_ODTIMB        10
CTR_OBLTCL         0
CTR_OBLTHO        10
CTR_OBLDLI        10
CTR_OBLTVI        10
CTR_RATISS         0
EXE_EXERCI         0
TVA_CHAFF6      1963
TVA_CHAFF7      1963
TVA_CHAF10      1963
TVA_CHAF12      1963
TVA_CHAF15      1963
TVA_CHAF18      1963
TVA_CHAF22      1963
TVA_CHAF29      1963
TVA_TOTDUE      1963
TVA_MOIDEB      1963
TVA_CRDINI      1963
TVA_BASIMB      1963
TVA_DEDIMB      1963
TVA_BASEQL      1963
TVA_DEDEQL      1963
TVA_BASEQI      1963
TVA_DEDEQI      1963
TVA_BASAUL      1963
TVA_DEDAUL      1963
TVA_BASAUI      1963
TVA_DEDAUI      1963
TVA_BASRSM      1963
TVA_DEDRSM   

In [23]:
train.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,TVA_BASEQL,TVA_DEDEQL,TVA_BASEQI,TVA_DEDEQI,TVA_BASAUL,TVA_DEDAUL,TVA_BASAUI,TVA_DEDAUI,TVA_BASRSM,TVA_DEDRSM,TVA_RSNRES,TVA_TRSPOR,TVA_DEDREG,TVA_RESTIT,TVA_MNTPAY,TVA_MOIFIN,TVA_CRDFIN,TVA_ACHSUS,TVA_ACHEXO,TVA_ACHNDD,TVA_CAFEXP,TVA_CAFSUS,TVA_CAFEXO,TVA_AACHAB,TVA_CRDBLQ,TVA_CRDRES,TVA_CRDRMP,TVA_CRDRMC,TVA_MACHN,TVA_MCESQ,TVA_MTVLSM,TVA_TXVLSM,TVA_MTALSM,TVA_TXALSM,TVA_MTORDTVA,TVA_MTDJRTVA,TVA_MTDJRTAX,TVA_MTRESTAX,TVA_CA072018,TVA_CA132018,TVA_CA192018,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,target,id_freq
0,49,14185,2,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,612500.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123608.0,9896.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6412354.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58912000.0,0.0,2530630.0,0.0,2013,13.575957,0.00004
1,221,6032,2,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3425295.0,0.0,10924985.0,0.0,2013,11.692894,0.00004
2,181,8907,2,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18861497.0,2209299.0,0.0,0.0,3566839.0,1783419.0,0.0,0.0,0.0,0.0,31150.0,12.0,192751.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2015,0.000000,0.00004
3,252,5783,1,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,0.0,0.0,0.0,0.0,9083122.0,1634962.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1680387.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2015,16.237871,0.00004
4,234,1477,2,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,1521555.0,273879.0,0.0,0.0,4564665.0,821637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,224442.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21604529.0,0.0,0.0,0.0,2015,13.399983,0.00004


In [25]:
def fill_na(df):
    for col in df.columns:
        df[col].fillna(df[col].mean(), inplace=True)
    return df

In [26]:
train = fill_na(train)
test = fill_na(test)

In [30]:
test.isna().sum()

BCT_CODBUR      0
CTR_MATFIS      0
CTR_CATEGO_X    0
FJU_CODFJU      0
CTR_CESSAT      0
ACT_CODACT      0
CTR_OBLDIR      0
CTR_OBLACP      0
CTR_OBLRES      0
CTR_OBLFOP      0
CTR_OBLTFP      0
CTR_OBLDCO      0
CTR_OBLTVA      0
CTR_OFODEC      0
CTR_OFODEP      0
CTR_OFODET      0
CTR_OBLAUT      0
CTR_OBLASS      0
CTR_ODTIMB      0
CTR_OBLTCL      0
CTR_OBLTHO      0
CTR_OBLDLI      0
CTR_OBLTVI      0
CTR_RATISS      0
EXE_EXERCI      0
TVA_CHAFF6      0
TVA_CHAFF7      0
TVA_CHAF10      0
TVA_CHAF12      0
TVA_CHAF15      0
TVA_CHAF18      0
TVA_CHAF22      0
TVA_CHAF29      0
TVA_TOTDUE      0
TVA_MOIDEB      0
TVA_CRDINI      0
TVA_BASIMB      0
TVA_DEDIMB      0
TVA_BASEQL      0
TVA_DEDEQL      0
TVA_BASEQI      0
TVA_DEDEQI      0
TVA_BASAUL      0
TVA_DEDAUL      0
TVA_BASAUI      0
TVA_DEDAUI      0
TVA_BASRSM      0
TVA_DEDRSM      0
TVA_RSNRES      0
TVA_TRSPOR      0
TVA_DEDREG      0
TVA_RESTIT      0
TVA_MNTPAY      0
TVA_MOIFIN      0
TVA_CRDFIN      0
TVA_ACHSUS

In [31]:
train.shape

(25042, 87)

In [33]:
X = train.drop('target', axis=1)
y = train['target']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [35]:
def rmse(val_test, val_predict):
    acc = np.sqrt(mean_squared_error(val_test, val_predict))
    return acc

In [39]:
cat_model = CatBoostRegressor(learning_rate=0.15, iterations=115, depth=15)
kf = KFold(n_splits=10, shuffle=True, random_state=30)

In [40]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    cat_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500, use_best_model=True)
    scores.append(rmse(y_test, cat_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
0:	learn: 6.8751067	test: 6.8751067	test1: 6.7297392	best: 6.7297392 (0)	total: 6.46s	remaining: 12m 16s
1:	learn: 6.6912901	test: 6.6912901	test1: 6.5978827	best: 6.5978827 (1)	total: 12.6s	remaining: 11m 52s
2:	learn: 6.5350644	test: 6.5350644	test1: 6.4732373	best: 6.4732373 (2)	total: 18.8s	remaining: 11m 41s
3:	learn: 6.4067441	test: 6.4067441	test1: 6.3813409	best: 6.3813409 (3)	total: 25s	remaining: 11m 33s
4:	learn: 6.2698676	test: 6.2698676	test1: 6.3265076	best: 6.3265076 (4)	total: 31.2s	remaining: 11m 25s
5:	learn: 6.1650289	test: 6.1650289	test1: 6.2713744	best: 6.2713744 (5)	total: 37.4s	remaining: 11m 19s
6:	learn: 6.0811608	test: 6.0811608	test1: 6.2196135	best: 6.2196135 (6)	total: 43.5s	remaining: 11m 11s
7:	learn: 5.9954596	test: 5.9954596	test1: 6.1735157	best: 6.1735157 (7)	total: 49.7s	remaining: 11m 4s
8:	learn: 5.9139587	test: 5.9139587	test1: 6.1437041	best: 6.1437041 (8)	total: 55.9s	remaining: 10m 57s
9:	learn: 5.8452101	test: 5

77:	learn: 3.9340782	test: 3.9340782	test1: 5.7439757	best: 5.7416915 (74)	total: 8m 10s	remaining: 3m 52s
78:	learn: 3.9150404	test: 3.9150404	test1: 5.7451117	best: 5.7416915 (74)	total: 8m 16s	remaining: 3m 46s
79:	learn: 3.8965788	test: 3.8965788	test1: 5.7430968	best: 5.7416915 (74)	total: 8m 22s	remaining: 3m 39s
80:	learn: 3.8716718	test: 3.8716718	test1: 5.7400178	best: 5.7400178 (80)	total: 8m 28s	remaining: 3m 33s
81:	learn: 3.8665039	test: 3.8665039	test1: 5.7392112	best: 5.7392112 (81)	total: 8m 34s	remaining: 3m 27s
82:	learn: 3.8547836	test: 3.8547836	test1: 5.7392146	best: 5.7392112 (81)	total: 8m 40s	remaining: 3m 20s
83:	learn: 3.8482580	test: 3.8482580	test1: 5.7370005	best: 5.7370005 (83)	total: 8m 46s	remaining: 3m 14s
84:	learn: 3.8136535	test: 3.8136535	test1: 5.7339479	best: 5.7339479 (84)	total: 8m 52s	remaining: 3m 8s
85:	learn: 3.8097987	test: 3.8097987	test1: 5.7331593	best: 5.7331593 (85)	total: 8m 58s	remaining: 3m 1s
86:	learn: 3.7928219	test: 3.7928219	te

38:	learn: 4.6696485	test: 4.6696485	test1: 6.0406582	best: 6.0406582 (38)	total: 3m 57s	remaining: 7m 42s
39:	learn: 4.6537159	test: 4.6537159	test1: 6.0372021	best: 6.0372021 (39)	total: 4m 3s	remaining: 7m 36s
40:	learn: 4.6144582	test: 4.6144582	test1: 6.0317952	best: 6.0317952 (40)	total: 4m 9s	remaining: 7m 30s
41:	learn: 4.5915457	test: 4.5915457	test1: 6.0314234	best: 6.0314234 (41)	total: 4m 15s	remaining: 7m 24s
42:	learn: 4.5697675	test: 4.5697675	test1: 6.0292524	best: 6.0292524 (42)	total: 4m 22s	remaining: 7m 18s
43:	learn: 4.5613757	test: 4.5613757	test1: 6.0287010	best: 6.0287010 (43)	total: 4m 28s	remaining: 7m 12s
44:	learn: 4.5378334	test: 4.5378334	test1: 6.0274393	best: 6.0274393 (44)	total: 4m 34s	remaining: 7m 6s
45:	learn: 4.5175395	test: 4.5175395	test1: 6.0217926	best: 6.0217926 (45)	total: 4m 40s	remaining: 7m
46:	learn: 4.5000465	test: 4.5000465	test1: 6.0167963	best: 6.0167963 (46)	total: 4m 46s	remaining: 6m 54s
47:	learn: 4.4864631	test: 4.4864631	test1: 

5.932865940455791
============fold2=============
0:	learn: 6.8539537	test: 6.8539537	test1: 6.9445540	best: 6.9445540 (0)	total: 6.01s	remaining: 11m 25s
1:	learn: 6.6733861	test: 6.6733861	test1: 6.8038208	best: 6.8038208 (1)	total: 12.1s	remaining: 11m 22s
2:	learn: 6.5193754	test: 6.5193754	test1: 6.7005701	best: 6.7005701 (2)	total: 18.1s	remaining: 11m 17s
3:	learn: 6.3821137	test: 6.3821137	test1: 6.5831807	best: 6.5831807 (3)	total: 24.2s	remaining: 11m 12s
4:	learn: 6.2483155	test: 6.2483155	test1: 6.5266172	best: 6.5266172 (4)	total: 30.4s	remaining: 11m 9s
5:	learn: 6.1326238	test: 6.1326238	test1: 6.4714404	best: 6.4714404 (5)	total: 36.5s	remaining: 11m 3s
6:	learn: 6.0349615	test: 6.0349615	test1: 6.4244516	best: 6.4244516 (6)	total: 42.6s	remaining: 10m 57s
7:	learn: 5.9440291	test: 5.9440291	test1: 6.3670702	best: 6.3670702 (7)	total: 48.7s	remaining: 10m 51s
8:	learn: 5.8542666	test: 5.8542666	test1: 6.3228954	best: 6.3228954 (8)	total: 54.8s	remaining: 10m 45s
9:	learn

77:	learn: 3.8571631	test: 3.8571631	test1: 5.9233235	best: 5.9233235 (77)	total: 7m 59s	remaining: 3m 47s
78:	learn: 3.8491352	test: 3.8491352	test1: 5.9230336	best: 5.9230336 (78)	total: 8m 5s	remaining: 3m 41s
79:	learn: 3.8323515	test: 3.8323515	test1: 5.9220230	best: 5.9220230 (79)	total: 8m 12s	remaining: 3m 35s
80:	learn: 3.8286134	test: 3.8286134	test1: 5.9210686	best: 5.9210686 (80)	total: 8m 18s	remaining: 3m 29s
81:	learn: 3.8073498	test: 3.8073498	test1: 5.9203206	best: 5.9203206 (81)	total: 8m 24s	remaining: 3m 22s
82:	learn: 3.8051508	test: 3.8051508	test1: 5.9195123	best: 5.9195123 (82)	total: 8m 30s	remaining: 3m 16s
83:	learn: 3.7940651	test: 3.7940651	test1: 5.9202175	best: 5.9195123 (82)	total: 8m 36s	remaining: 3m 10s
84:	learn: 3.7659979	test: 3.7659979	test1: 5.9201312	best: 5.9195123 (82)	total: 8m 42s	remaining: 3m 4s
85:	learn: 3.7627374	test: 3.7627374	test1: 5.9188428	best: 5.9188428 (85)	total: 8m 48s	remaining: 2m 58s
86:	learn: 3.7449719	test: 3.7449719	te

...

38:	learn: 4.7265137	test: 4.7265137	test1: 5.9902274	best: 5.9902274 (38)	total: 3m 57s	remaining: 7m 41s
39:	learn: 4.7072871	test: 4.7072871	test1: 5.9855903	best: 5.9855903 (39)	total: 4m 3s	remaining: 7m 35s
40:	learn: 4.6623800	test: 4.6623800	test1: 5.9920029	best: 5.9855903 (39)	total: 4m 9s	remaining: 7m 30s
41:	learn: 4.6514120	test: 4.6514120	test1: 5.9881460	best: 5.9855903 (39)	total: 4m 15s	remaining: 7m 24s
42:	learn: 4.6408530	test: 4.6408530	test1: 5.9836619	best: 5.9836619 (42)	total: 4m 21s	remaining: 7m 17s
43:	learn: 4.6312762	test: 4.6312762	test1: 5.9802600	best: 5.9802600 (43)	total: 4m 27s	remaining: 7m 11s
44:	learn: 4.6072083	test: 4.6072083	test1: 5.9798156	best: 5.9798156 (44)	total: 4m 33s	remaining: 7m 5s
45:	learn: 4.5709515	test: 4.5709515	test1: 5.9789394	best: 5.9789394 (45)	total: 4m 39s	remaining: 6m 59s
46:	learn: 4.5552312	test: 4.5552312	test1: 5.9780909	best: 5.9780909 (46)	total: 4m 45s	remaining: 6m 53s
47:	learn: 4.5344266	test: 4.5344266	tes

5.8580760858799765
============fold8=============
0:	learn: 6.8546227	test: 6.8546227	test1: 6.9626578	best: 6.9626578 (0)	total: 5.95s	remaining: 11m 17s
1:	learn: 6.6676782	test: 6.6676782	test1: 6.8427154	best: 6.8427154 (1)	total: 11.9s	remaining: 11m 14s
2:	learn: 6.5086705	test: 6.5086705	test1: 6.7267899	best: 6.7267899 (2)	total: 18s	remaining: 11m 11s
3:	learn: 6.3755552	test: 6.3755552	test1: 6.6227621	best: 6.6227621 (3)	total: 24s	remaining: 11m 5s
4:	learn: 6.2617284	test: 6.2617284	test1: 6.5691766	best: 6.5691766 (4)	total: 30.2s	remaining: 11m 4s
5:	learn: 6.1650027	test: 6.1650027	test1: 6.5246043	best: 6.5246043 (5)	total: 36.2s	remaining: 10m 58s
6:	learn: 6.0751238	test: 6.0751238	test1: 6.4707925	best: 6.4707925 (6)	total: 42.2s	remaining: 10m 51s
7:	learn: 5.9841603	test: 5.9841603	test1: 6.4280091	best: 6.4280091 (7)	total: 48.2s	remaining: 10m 44s
8:	learn: 5.9030234	test: 5.9030234	test1: 6.4087428	best: 6.4087428 (8)	total: 54.3s	remaining: 10m 39s
9:	learn: 5

77:	learn: 3.9583215	test: 3.9583215	test1: 6.0634284	best: 6.0634284 (77)	total: 7m 41s	remaining: 3m 38s
78:	learn: 3.9358433	test: 3.9358433	test1: 6.0649071	best: 6.0634284 (77)	total: 7m 47s	remaining: 3m 33s
79:	learn: 3.9134870	test: 3.9134870	test1: 6.0665919	best: 6.0634284 (77)	total: 7m 53s	remaining: 3m 27s
80:	learn: 3.8880907	test: 3.8880907	test1: 6.0659342	best: 6.0634284 (77)	total: 7m 59s	remaining: 3m 21s
81:	learn: 3.8811827	test: 3.8811827	test1: 6.0634998	best: 6.0634284 (77)	total: 8m 5s	remaining: 3m 15s
82:	learn: 3.8580198	test: 3.8580198	test1: 6.0633935	best: 6.0633935 (82)	total: 8m 11s	remaining: 3m 9s
83:	learn: 3.8436813	test: 3.8436813	test1: 6.0645162	best: 6.0633935 (82)	total: 8m 17s	remaining: 3m 3s
84:	learn: 3.8293480	test: 3.8293480	test1: 6.0652594	best: 6.0633935 (82)	total: 8m 23s	remaining: 2m 57s
85:	learn: 3.8082034	test: 3.8082034	test1: 6.0625708	best: 6.0625708 (85)	total: 8m 30s	remaining: 2m 52s
86:	learn: 3.7942859	test: 3.7942859	tes

38:	learn: 4.8856016	test: 4.8856016	test1: 6.0222639	best: 6.0222639 (38)	total: 3m 57s	remaining: 7m 41s
39:	learn: 4.8766270	test: 4.8766270	test1: 6.0197289	best: 6.0197289 (39)	total: 4m 3s	remaining: 7m 35s
40:	learn: 4.8586834	test: 4.8586834	test1: 6.0122155	best: 6.0122155 (40)	total: 4m 9s	remaining: 7m 29s
41:	learn: 4.8450232	test: 4.8450232	test1: 6.0073224	best: 6.0073224 (41)	total: 4m 15s	remaining: 7m 23s
42:	learn: 4.8287774	test: 4.8287774	test1: 6.0058943	best: 6.0058943 (42)	total: 4m 21s	remaining: 7m 17s
43:	learn: 4.8117082	test: 4.8117082	test1: 5.9972915	best: 5.9972915 (43)	total: 4m 27s	remaining: 7m 11s
44:	learn: 4.7934108	test: 4.7934108	test1: 5.9956970	best: 5.9956970 (44)	total: 4m 33s	remaining: 7m 5s
45:	learn: 4.7728393	test: 4.7728393	test1: 5.9931101	best: 5.9931101 (45)	total: 4m 39s	remaining: 6m 59s
46:	learn: 4.7388153	test: 4.7388153	test1: 5.9893216	best: 5.9893216 (46)	total: 4m 45s	remaining: 6m 53s
47:	learn: 4.7017543	test: 4.7017543	tes

5.878981311088915
[5.706087851522418, 5.932865940455791, 5.9122278167508675, 5.956966467754009, 5.736580434072922, 5.945894987148107, 5.769887131468767, 5.8580760858799765, 6.04115469728956, 5.878981311088915]


In [41]:
cat_final = cat_model.predict(test)

In [42]:
submit['target'] = cat_final

In [43]:
submit.to_csv('kfold.csv', index=False)

In [ ]:
#gird search

In [62]:
rfc_model = RandomForestRegressor(n_estimators = 210, min_samples_split = 18, max_depth = 26)

In [63]:
rfc_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=26, min_samples_split=18, n_estimators=210)

In [64]:
rfc_pred = rfc_model.predict(X_test)

In [65]:
np.sqrt(mean_squared_error(y_test, rfc_pred))

5.8757305314687684

In [66]:
rfc_final = rfc_model.predict(test)

In [67]:
submit['target'] = rfc_final

In [68]:
submit.to_csv('krfc.csv', index=False)

In [69]:
rfc_kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [70]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    rfc_model.fit(X_train, y_train, )
    scores.append(rmse(y_test, rfc_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
5.745461631783786
============fold1=============
5.999441153331977
============fold2=============
5.952092616309021
============fold3=============
5.9881308471402575
============fold4=============
5.759379477214732
============fold5=============
5.943928969872281
============fold6=============
5.709779591240481
============fold7=============
5.897248109135784
============fold8=============
5.988026067480111
============fold9=============
5.867396212455925
[5.745461631783786, 5.999441153331977, 5.952092616309021, 5.9881308471402575, 5.759379477214732, 5.943928969872281, 5.709779591240481, 5.897248109135784, 5.988026067480111, 5.867396212455925]


In [72]:
rfc = rfc_model.predict(test)

In [73]:
submit['target'] = rfc

In [74]:
submit.to_csv('krfc2.csv', index=False)

In [57]:
parameters = {'max_depth': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30], 'min_samples_split': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'n_estimators': [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230]}

In [58]:
rand_search = RandomizedSearchCV(estimator=rfc_model, param_distributions=parameters, cv=2, n_jobs=-1)

In [59]:
rand_search.fit(X, y)

RandomizedSearchCV(cv=2,
                   estimator=RandomForestRegressor(max_depth=5,
                                                   min_samples_split=7,
                                                   n_estimators=200),
                   n_jobs=-1,
                   param_distributions={'max_depth': [6, 7, 8, 9, 10, 11, 12,
                                                      13, 14, 15, 16, 17, 18,
                                                      19, 20, 21, 22, 23, 24,
                                                      25, 25, 26, 27, 28, 29,
                                                      30],
                                        'min_samples_split': [10, 11, 12, 13,
                                                              14, 15, 16, 17,
                                                              18, 19, 20],
                                        'n_estimators': [100, 105, 110, 115,
                                                         120

In [60]:
print('Result:')
print('\n the best estimator across all searched parmas: ', rand_search.best_estimator_)
print('\n the best score across all searched params: ', rand_search.best_score_)
print('\n the best parameters acress al best searched params: ', rand_search.best_params_)

Result:

 the best estimator across all searched parmas:  RandomForestRegressor(max_depth=26, min_samples_split=18, n_estimators=210)

 the best score across all searched params:  0.27707927441221947

 the best parameters acress al best searched params:  {'n_estimators': 210, 'min_samples_split': 18, 'max_depth': 26}


In [ ]:
cat_model = CatBoostRegressor(learning_rate=0.15, iterations=115, depth=15)

In [ ]:
cat_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500, use_best_model=True)

In [ ]:
cat_final = cat_model.predict(test)

In [ ]:
submit['target'] = cat_final

In [ ]:
submit.to_csv('rscv.csv', index=False)

In [76]:
parameters = {'max_depth': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30], 'learning_rate': [0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, .020], 'n_estimators': [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230], 'reg_lambda': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]}

In [77]:
rand_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=parameters, cv=2, n_jobs=-1)

In [78]:
rand_search.fit(X, y)

RandomizedSearchCV(cv=2,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=0.1,
                                          max_delta_step=None, max_depth=10,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=150, n_jo...
                   param_distributions={'learning_rate': [0.05, 0.06, 0.07,
                                                          0.08, 0.09, 0.1, 0.11,
                                 

In [79]:
print('Result:')
print('\n the best estimator across all searched parmas: ', rand_search.best_estimator_)
print('\n the best score across all searched params: ', rand_search.best_score_)
print('\n the best parameters acress al best searched params: ', rand_search.best_params_)

Result:

 the best estimator across all searched parmas:  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=17,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=135, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=12, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

 the best score across all searched params:  0.2982213147239545

 the best parameters acress al best searched params:  {'reg_lambda': 12, 'n_estimators': 135, 'max_depth': 17, 'learning_rate': 0.05}


In [80]:
xgb_model = XGBRegressor(learning_rate =0.05, n_estimators=135, max_depth=17, reg_lambda=12)

In [81]:
kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [82]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    xgb_model.fit(X_train, y_train, eval_metric='rmse', eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500)
    scores.append(rmse(y_test, xgb_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
[0]	validation_0-rmse:12.79973	validation_1-rmse:12.96288
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 500 rounds.
[1]	validation_0-rmse:12.30147	validation_1-rmse:12.46744
[2]	validation_0-rmse:11.83088	validation_1-rmse:12.00325
[3]	validation_0-rmse:11.38845	validation_1-rmse:11.56380
[4]	validation_0-rmse:10.96970	validation_1-rmse:11.15164
[5]	validation_0-rmse:10.57535	validation_1-rmse:10.77070
[6]	validation_0-rmse:10.20564	validation_1-rmse:10.41003
[7]	validation_0-rmse:9.85387	validation_1-rmse:10.07459
[8]	validation_0-rmse:9.52174	validation_1-rmse:9.75794
[9]	validation_0-rmse:9.20836	validation_1-rmse:9.46362
[10]	validation_0-rmse:8.91495	validation_1-rmse:9.18663
[11]	validation_0-rmse:8.63651	validation_1-rmse:8.92467
[12]	validation_0-rmse:8.37281	validation_1-rmse:8.68214
[13]	validation_0-rmse:8.12277	validation_1-rmse:8.45867
[14]	v

[1]	validation_0-rmse:12.29976	validation_1-rmse:12.48273
[2]	validation_0-rmse:11.82826	validation_1-rmse:12.02976
[3]	validation_0-rmse:11.38310	validation_1-rmse:11.61021
[4]	validation_0-rmse:10.96328	validation_1-rmse:11.21613
[5]	validation_0-rmse:10.56653	validation_1-rmse:10.85022
[6]	validation_0-rmse:10.19070	validation_1-rmse:10.50532
[7]	validation_0-rmse:9.83796	validation_1-rmse:10.18272
[8]	validation_0-rmse:9.50712	validation_1-rmse:9.88384
[9]	validation_0-rmse:9.19383	validation_1-rmse:9.60353
[10]	validation_0-rmse:8.89538	validation_1-rmse:9.33736
[11]	validation_0-rmse:8.61512	validation_1-rmse:9.09468
[12]	validation_0-rmse:8.34980	validation_1-rmse:8.86319
[13]	validation_0-rmse:8.10121	validation_1-rmse:8.65004
[14]	validation_0-rmse:7.86438	validation_1-rmse:8.45168
[15]	validation_0-rmse:7.64317	validation_1-rmse:8.26619
[16]	validation_0-rmse:7.43354	validation_1-rmse:8.09625
[17]	validation_0-rmse:7.23313	validation_1-rmse:7.93587
[18]	validation_0-rmse:7.04

[6]	validation_0-rmse:10.19700	validation_1-rmse:10.34554
[7]	validation_0-rmse:9.84346	validation_1-rmse:10.01996
[8]	validation_0-rmse:9.50971	validation_1-rmse:9.71571
[9]	validation_0-rmse:9.19794	validation_1-rmse:9.43401
[10]	validation_0-rmse:8.90170	validation_1-rmse:9.16908
[11]	validation_0-rmse:8.62177	validation_1-rmse:8.92551
[12]	validation_0-rmse:8.35492	validation_1-rmse:8.69663
[13]	validation_0-rmse:8.10455	validation_1-rmse:8.47777
[14]	validation_0-rmse:7.86919	validation_1-rmse:8.27557
[15]	validation_0-rmse:7.64537	validation_1-rmse:8.09244
[16]	validation_0-rmse:7.43548	validation_1-rmse:7.91852
[17]	validation_0-rmse:7.23473	validation_1-rmse:7.76265
[18]	validation_0-rmse:7.04690	validation_1-rmse:7.61441
[19]	validation_0-rmse:6.86626	validation_1-rmse:7.47934
[20]	validation_0-rmse:6.69231	validation_1-rmse:7.35781
[21]	validation_0-rmse:6.52593	validation_1-rmse:7.24038
[22]	validation_0-rmse:6.37116	validation_1-rmse:7.12982
[23]	validation_0-rmse:6.22573	v

[11]	validation_0-rmse:8.62383	validation_1-rmse:8.83139
[12]	validation_0-rmse:8.35777	validation_1-rmse:8.61538
[13]	validation_0-rmse:8.11032	validation_1-rmse:8.41252
[14]	validation_0-rmse:7.87170	validation_1-rmse:8.22071
[15]	validation_0-rmse:7.64705	validation_1-rmse:8.04547
[16]	validation_0-rmse:7.43342	validation_1-rmse:7.88287
[17]	validation_0-rmse:7.23191	validation_1-rmse:7.73138
[18]	validation_0-rmse:7.04611	validation_1-rmse:7.59508
[19]	validation_0-rmse:6.86501	validation_1-rmse:7.46692
[20]	validation_0-rmse:6.69563	validation_1-rmse:7.34952
[21]	validation_0-rmse:6.53451	validation_1-rmse:7.24642
[22]	validation_0-rmse:6.38264	validation_1-rmse:7.14542
[23]	validation_0-rmse:6.23517	validation_1-rmse:7.05841
[24]	validation_0-rmse:6.09638	validation_1-rmse:6.97428
[25]	validation_0-rmse:5.96187	validation_1-rmse:6.89999
[26]	validation_0-rmse:5.82922	validation_1-rmse:6.83001
[27]	validation_0-rmse:5.70289	validation_1-rmse:6.76185
[28]	validation_0-rmse:5.58462	

[16]	validation_0-rmse:7.44023	validation_1-rmse:7.95172
[17]	validation_0-rmse:7.24027	validation_1-rmse:7.79362
[18]	validation_0-rmse:7.05099	validation_1-rmse:7.64417
[19]	validation_0-rmse:6.87122	validation_1-rmse:7.50757
[20]	validation_0-rmse:6.69925	validation_1-rmse:7.37757
[21]	validation_0-rmse:6.53845	validation_1-rmse:7.25778
[22]	validation_0-rmse:6.38054	validation_1-rmse:7.14853
[23]	validation_0-rmse:6.23173	validation_1-rmse:7.04588
[24]	validation_0-rmse:6.09223	validation_1-rmse:6.95119
[25]	validation_0-rmse:5.96000	validation_1-rmse:6.86246
[26]	validation_0-rmse:5.83397	validation_1-rmse:6.78165
[27]	validation_0-rmse:5.71281	validation_1-rmse:6.70978
[28]	validation_0-rmse:5.60049	validation_1-rmse:6.64665
[29]	validation_0-rmse:5.49181	validation_1-rmse:6.58396
[30]	validation_0-rmse:5.39107	validation_1-rmse:6.52841
[31]	validation_0-rmse:5.28384	validation_1-rmse:6.47579
[32]	validation_0-rmse:5.18379	validation_1-rmse:6.42997
[33]	validation_0-rmse:5.08777	

...

[46]	validation_0-rmse:4.17440	validation_1-rmse:6.02148
[47]	validation_0-rmse:4.13194	validation_1-rmse:6.00930
[48]	validation_0-rmse:4.08427	validation_1-rmse:5.99346
[49]	validation_0-rmse:4.04132	validation_1-rmse:5.98205
[50]	validation_0-rmse:3.99420	validation_1-rmse:5.96643
[51]	validation_0-rmse:3.95801	validation_1-rmse:5.95526
[52]	validation_0-rmse:3.92661	validation_1-rmse:5.94254
[53]	validation_0-rmse:3.89829	validation_1-rmse:5.93470
[54]	validation_0-rmse:3.87032	validation_1-rmse:5.92717
[55]	validation_0-rmse:3.84924	validation_1-rmse:5.92113
[56]	validation_0-rmse:3.83082	validation_1-rmse:5.91742
[57]	validation_0-rmse:3.80669	validation_1-rmse:5.90932
[58]	validation_0-rmse:3.78532	validation_1-rmse:5.90381
[59]	validation_0-rmse:3.76844	validation_1-rmse:5.89645
[60]	validation_0-rmse:3.74278	validation_1-rmse:5.89192
[61]	validation_0-rmse:3.72572	validation_1-rmse:5.88768
[62]	validation_0-rmse:3.71191	validation_1-rmse:5.88273
[63]	validation_0-rmse:3.69610	

[51]	validation_0-rmse:3.93819	validation_1-rmse:5.81926
[52]	validation_0-rmse:3.90738	validation_1-rmse:5.80832
[53]	validation_0-rmse:3.87981	validation_1-rmse:5.79924
[54]	validation_0-rmse:3.84827	validation_1-rmse:5.78876
[55]	validation_0-rmse:3.82254	validation_1-rmse:5.77863
[56]	validation_0-rmse:3.80085	validation_1-rmse:5.76935
[57]	validation_0-rmse:3.77933	validation_1-rmse:5.76056
[58]	validation_0-rmse:3.74988	validation_1-rmse:5.75169
[59]	validation_0-rmse:3.72791	validation_1-rmse:5.74541
[60]	validation_0-rmse:3.70387	validation_1-rmse:5.73960
[61]	validation_0-rmse:3.68493	validation_1-rmse:5.73501
[62]	validation_0-rmse:3.65829	validation_1-rmse:5.72771
[63]	validation_0-rmse:3.63889	validation_1-rmse:5.72221
[64]	validation_0-rmse:3.61872	validation_1-rmse:5.72023
[65]	validation_0-rmse:3.60179	validation_1-rmse:5.71650
[66]	validation_0-rmse:3.58289	validation_1-rmse:5.71184
[67]	validation_0-rmse:3.56668	validation_1-rmse:5.70715
[68]	validation_0-rmse:3.54746	

[56]	validation_0-rmse:3.79844	validation_1-rmse:6.01952
[57]	validation_0-rmse:3.77612	validation_1-rmse:6.01401
[58]	validation_0-rmse:3.74756	validation_1-rmse:6.00785
[59]	validation_0-rmse:3.72227	validation_1-rmse:6.00115
[60]	validation_0-rmse:3.70639	validation_1-rmse:5.99550
[61]	validation_0-rmse:3.68708	validation_1-rmse:5.99126
[62]	validation_0-rmse:3.66605	validation_1-rmse:5.98548
[63]	validation_0-rmse:3.64266	validation_1-rmse:5.97917
[64]	validation_0-rmse:3.63036	validation_1-rmse:5.97738
[65]	validation_0-rmse:3.60840	validation_1-rmse:5.97223
[66]	validation_0-rmse:3.58412	validation_1-rmse:5.96892
[67]	validation_0-rmse:3.57478	validation_1-rmse:5.96634
[68]	validation_0-rmse:3.55299	validation_1-rmse:5.96391
[69]	validation_0-rmse:3.54363	validation_1-rmse:5.96240
[70]	validation_0-rmse:3.53411	validation_1-rmse:5.96082
[71]	validation_0-rmse:3.51650	validation_1-rmse:5.95854
[72]	validation_0-rmse:3.49047	validation_1-rmse:5.95548
[73]	validation_0-rmse:3.48258	

[61]	validation_0-rmse:3.66264	validation_1-rmse:5.82484
[62]	validation_0-rmse:3.64951	validation_1-rmse:5.82183
[63]	validation_0-rmse:3.63529	validation_1-rmse:5.81913
[64]	validation_0-rmse:3.62217	validation_1-rmse:5.81691
[65]	validation_0-rmse:3.60436	validation_1-rmse:5.81365
[66]	validation_0-rmse:3.58600	validation_1-rmse:5.80891
[67]	validation_0-rmse:3.57402	validation_1-rmse:5.80592
[68]	validation_0-rmse:3.55795	validation_1-rmse:5.80398
[69]	validation_0-rmse:3.54788	validation_1-rmse:5.80097
[70]	validation_0-rmse:3.52454	validation_1-rmse:5.79774
[71]	validation_0-rmse:3.50938	validation_1-rmse:5.79491
[72]	validation_0-rmse:3.49931	validation_1-rmse:5.79170
[73]	validation_0-rmse:3.49115	validation_1-rmse:5.78998
[74]	validation_0-rmse:3.47783	validation_1-rmse:5.78814
[75]	validation_0-rmse:3.46441	validation_1-rmse:5.78559
[76]	validation_0-rmse:3.45822	validation_1-rmse:5.78559
[77]	validation_0-rmse:3.44531	validation_1-rmse:5.78418
[78]	validation_0-rmse:3.42949	

[66]	validation_0-rmse:3.66146	validation_1-rmse:5.75742
[67]	validation_0-rmse:3.65190	validation_1-rmse:5.75557
[68]	validation_0-rmse:3.63917	validation_1-rmse:5.75378
[69]	validation_0-rmse:3.62705	validation_1-rmse:5.75059
[70]	validation_0-rmse:3.61685	validation_1-rmse:5.75007
[71]	validation_0-rmse:3.60364	validation_1-rmse:5.74852
[72]	validation_0-rmse:3.59472	validation_1-rmse:5.74676
[73]	validation_0-rmse:3.58326	validation_1-rmse:5.74478
[74]	validation_0-rmse:3.56846	validation_1-rmse:5.74256
[75]	validation_0-rmse:3.55881	validation_1-rmse:5.74246
[76]	validation_0-rmse:3.55027	validation_1-rmse:5.74120
[77]	validation_0-rmse:3.52911	validation_1-rmse:5.73983
[78]	validation_0-rmse:3.50663	validation_1-rmse:5.73687
[79]	validation_0-rmse:3.49367	validation_1-rmse:5.73501
[80]	validation_0-rmse:3.47004	validation_1-rmse:5.73102
[81]	validation_0-rmse:3.45680	validation_1-rmse:5.72867
[82]	validation_0-rmse:3.44541	validation_1-rmse:5.72555
[83]	validation_0-rmse:3.43815	

In [83]:
xgb_final = xgb_model.predict(test)

In [84]:
submit['target'] = xgb_final

In [85]:
submit.to_csv('kxgb.csv', index=False)

In [95]:
knn_model = KNeighborsRegressor(n_neighbors=5)

In [96]:
kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [97]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    knn_model.fit(X_train, y_train)
    scores.append(rmse(y_test, knn_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
6.7524488304535755
============fold1=============
7.064823702734056
============fold2=============
6.870061921822632
============fold3=============
6.8984367112109615
============fold4=============
6.70932765853418
============fold5=============
7.03435736291269
============fold6=============
6.752902287236639
============fold7=============
7.00005254666875
============fold8=============
6.869047339410071
============fold9=============
6.986486523751965
============fold10=============
6.834866677277323
============fold11=============
6.731517976398832
============fold12=============
6.927475140121996
============fold13=============
6.903136246093121
============fold14=============
6.998675586656031
[6.7524488304535755, 7.064823702734056, 6.870061921822632, 6.8984367112109615, 6.70932765853418, 7.03435736291269, 6.752902287236639, 7.00005254666875, 6.869047339410071, 6.986486523751965, 6.834866677277323, 6.731517976398832, 6.927475140121996, 6.903136246093

In [98]:
knn_final = knn_model.predict(test)

In [99]:
submit['target'] = knn_final

In [100]:
submit.to_csv('kknn.csv', index=False)

In [91]:
parameter = {'n_neighbors': [11,12,13,14,15,16,17,18,19,20]}

In [92]:
rand_search = RandomizedSearchCV(estimator=knn_model, param_distributions=parameter, cv=5, n_jobs=-1)

In [93]:
rand_search.fit(X, y)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
                   param_distributions={'n_neighbors': [11, 12, 13, 14, 15, 16,
                                                        17, 18, 19, 20]})

In [94]:
print('Result:')
print('\n the best estimator across all searched parmas: ', rand_search.best_estimator_)
print('\n the best score across all searched params: ', rand_search.best_score_)
print('\n the best parameters acress al best searched params: ', rand_search.best_params_)

Result:

 the best estimator across all searched parmas:  KNeighborsRegressor(n_neighbors=20)

 the best score across all searched params:  0.12404486549815708

 the best parameters acress al best searched params:  {'n_neighbors': 20}


In [102]:
parameters = {'num_leaves': [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50], 'max_depth': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30], 'learning_rate': [0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, .020], 'n_estimators': [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230], 'reg_lambda': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]}

In [106]:
rand_search = RandomizedSearchCV(estimator=lgb_model, param_distributions=parameters, cv=5, n_jobs=-1)

In [107]:
rand_search.fit(X, y)

RandomizedSearchCV(cv=5, estimator=LGBMRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.06, 0.07,
                                                          0.08, 0.09, 0.1, 0.11,
                                                          0.12, 0.13, 0.14,
                                                          0.15, 0.16, 0.17,
                                                          0.18, 0.19, 0.02],
                                        'max_depth': [6, 7, 8, 9, 10, 11, 12,
                                                      13, 14, 15, 16, 17, 18,
                                                      19, 20, 21, 22, 23, 24,
                                                      25, 25, 26, 27, 28, 29,
                                                      30],
                                        'n_estimators': [100, 105, 110, 115,
                                                         120, 125, 130, 135,
                          

In [108]:
print('Result:')
print('\n the best estimator across all searched parmas: ', rand_search.best_estimator_)
print('\n the best score across all searched params: ', rand_search.best_score_)
print('\n the best parameters acress all best searched params: ', rand_search.best_params_)

Result:

 the best estimator across all searched parmas:  LGBMRegressor(learning_rate=0.16, max_depth=19, n_estimators=195, num_leaves=50,
              reg_lambda=14)

 the best score across all searched params:  0.3146787590574238

 the best parameters acress all best searched params:  {'reg_lambda': 14, 'num_leaves': 50, 'n_estimators': 195, 'max_depth': 19, 'learning_rate': 0.16}


In [109]:
lgb_model = LGBMRegressor(reg_lambda=14, num_leaves=50, n_estimators=195, max_depth=19, learning_rate=0.16)

In [110]:
kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [115]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    lgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=500, eval_metric='rmse')
    scores.append(rmse(y_test, lgb_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
[1]	training's rmse: 6.83959	training's l2: 46.78	valid_1's rmse: 6.54582	valid_1's l2: 42.8477
Training until validation scores don't improve for 500 rounds
[2]	training's rmse: 6.64386	training's l2: 44.1409	valid_1's rmse: 6.36373	valid_1's l2: 40.4971
[3]	training's rmse: 6.48693	training's l2: 42.0803	valid_1's rmse: 6.21793	valid_1's l2: 38.6626
[4]	training's rmse: 6.36527	training's l2: 40.5166	valid_1's rmse: 6.11158	valid_1's l2: 37.3514
[5]	training's rmse: 6.26214	training's l2: 39.2144	valid_1's rmse: 6.03376	valid_1's l2: 36.4063
[6]	training's rmse: 6.18364	training's l2: 38.2373	valid_1's rmse: 5.96134	valid_1's l2: 35.5375
[7]	training's rmse: 6.11896	training's l2: 37.4417	valid_1's rmse: 5.90947	valid_1's l2: 34.9219
[8]	training's rmse: 6.0619	training's l2: 36.7466	valid_1's rmse: 5.87188	valid_1's l2: 34.4789
[9]	training's rmse: 6.01459	training's l2: 36.1753	valid_1's rmse: 5.84008	valid_1's l2: 34.1066
[10]	training's rmse: 5.9729

[86]	training's rmse: 4.91606	training's l2: 24.1676	valid_1's rmse: 5.57509	valid_1's l2: 31.0816
[87]	training's rmse: 4.90709	training's l2: 24.0795	valid_1's rmse: 5.57068	valid_1's l2: 31.0324
[88]	training's rmse: 4.90069	training's l2: 24.0167	valid_1's rmse: 5.56937	valid_1's l2: 31.0179
[89]	training's rmse: 4.8916	training's l2: 23.9277	valid_1's rmse: 5.56862	valid_1's l2: 31.0096
[90]	training's rmse: 4.88543	training's l2: 23.8674	valid_1's rmse: 5.56821	valid_1's l2: 31.005
[91]	training's rmse: 4.87724	training's l2: 23.7875	valid_1's rmse: 5.56748	valid_1's l2: 30.9968
[92]	training's rmse: 4.87058	training's l2: 23.7225	valid_1's rmse: 5.56836	valid_1's l2: 31.0067
[93]	training's rmse: 4.86222	training's l2: 23.6412	valid_1's rmse: 5.56712	valid_1's l2: 30.9928
[94]	training's rmse: 4.85014	training's l2: 23.5238	valid_1's rmse: 5.56631	valid_1's l2: 30.9838
[95]	training's rmse: 4.84322	training's l2: 23.4568	valid_1's rmse: 5.56728	valid_1's l2: 30.9946
[96]	trainin

[184]	training's rmse: 4.22028	training's l2: 17.8107	valid_1's rmse: 5.50542	valid_1's l2: 30.3097
[185]	training's rmse: 4.21474	training's l2: 17.764	valid_1's rmse: 5.50645	valid_1's l2: 30.321
[186]	training's rmse: 4.20602	training's l2: 17.6906	valid_1's rmse: 5.50718	valid_1's l2: 30.3291
[187]	training's rmse: 4.19847	training's l2: 17.6272	valid_1's rmse: 5.50271	valid_1's l2: 30.2798
[188]	training's rmse: 4.19267	training's l2: 17.5785	valid_1's rmse: 5.50401	valid_1's l2: 30.2941
[189]	training's rmse: 4.18559	training's l2: 17.5192	valid_1's rmse: 5.50405	valid_1's l2: 30.2945
[190]	training's rmse: 4.18239	training's l2: 17.4924	valid_1's rmse: 5.50473	valid_1's l2: 30.302
[191]	training's rmse: 4.17844	training's l2: 17.4594	valid_1's rmse: 5.50446	valid_1's l2: 30.2991
[192]	training's rmse: 4.17289	training's l2: 17.413	valid_1's rmse: 5.50321	valid_1's l2: 30.2854
[193]	training's rmse: 4.16673	training's l2: 17.3617	valid_1's rmse: 5.50524	valid_1's l2: 30.3077
[194

[75]	training's rmse: 5.00074	training's l2: 25.0074	valid_1's rmse: 6.02834	valid_1's l2: 36.3409
[76]	training's rmse: 4.99184	training's l2: 24.9185	valid_1's rmse: 6.0269	valid_1's l2: 36.3235
[77]	training's rmse: 4.98051	training's l2: 24.8055	valid_1's rmse: 6.02402	valid_1's l2: 36.2888
[78]	training's rmse: 4.9692	training's l2: 24.693	valid_1's rmse: 6.02327	valid_1's l2: 36.2798
[79]	training's rmse: 4.96348	training's l2: 24.6361	valid_1's rmse: 6.02186	valid_1's l2: 36.2628
[80]	training's rmse: 4.95569	training's l2: 24.5588	valid_1's rmse: 6.02101	valid_1's l2: 36.2525
[81]	training's rmse: 4.94584	training's l2: 24.4613	valid_1's rmse: 6.0148	valid_1's l2: 36.1778
[82]	training's rmse: 4.93824	training's l2: 24.3862	valid_1's rmse: 6.01577	valid_1's l2: 36.1895
[83]	training's rmse: 4.93191	training's l2: 24.3238	valid_1's rmse: 6.01279	valid_1's l2: 36.1537
[84]	training's rmse: 4.92443	training's l2: 24.25	valid_1's rmse: 6.01419	valid_1's l2: 36.1705
[85]	training's 

[164]	training's rmse: 4.30044	training's l2: 18.4938	valid_1's rmse: 5.91686	valid_1's l2: 35.0092
[165]	training's rmse: 4.29382	training's l2: 18.4369	valid_1's rmse: 5.91581	valid_1's l2: 34.9968
[166]	training's rmse: 4.28745	training's l2: 18.3823	valid_1's rmse: 5.91755	valid_1's l2: 35.0175
[167]	training's rmse: 4.27868	training's l2: 18.3071	valid_1's rmse: 5.91021	valid_1's l2: 34.9306
[168]	training's rmse: 4.27409	training's l2: 18.2678	valid_1's rmse: 5.91148	valid_1's l2: 34.9456
[169]	training's rmse: 4.26941	training's l2: 18.2278	valid_1's rmse: 5.90923	valid_1's l2: 34.919
[170]	training's rmse: 4.26099	training's l2: 18.156	valid_1's rmse: 5.90915	valid_1's l2: 34.918
[171]	training's rmse: 4.25733	training's l2: 18.1248	valid_1's rmse: 5.91012	valid_1's l2: 34.9295
[172]	training's rmse: 4.25101	training's l2: 18.0711	valid_1's rmse: 5.90989	valid_1's l2: 34.9268
[173]	training's rmse: 4.2473	training's l2: 18.0395	valid_1's rmse: 5.91115	valid_1's l2: 34.9417
[174

...

[46]	training's rmse: 5.28821	training's l2: 27.9652	valid_1's rmse: 5.93263	valid_1's l2: 35.1961
[47]	training's rmse: 5.27776	training's l2: 27.8547	valid_1's rmse: 5.93316	valid_1's l2: 35.2024
[48]	training's rmse: 5.2649	training's l2: 27.7192	valid_1's rmse: 5.92919	valid_1's l2: 35.1553
[49]	training's rmse: 5.25272	training's l2: 27.591	valid_1's rmse: 5.92391	valid_1's l2: 35.0927
[50]	training's rmse: 5.24099	training's l2: 27.468	valid_1's rmse: 5.92257	valid_1's l2: 35.0768
[51]	training's rmse: 5.2312	training's l2: 27.3655	valid_1's rmse: 5.92344	valid_1's l2: 35.0872
[52]	training's rmse: 5.22221	training's l2: 27.2714	valid_1's rmse: 5.92343	valid_1's l2: 35.087
[53]	training's rmse: 5.20831	training's l2: 27.1265	valid_1's rmse: 5.92596	valid_1's l2: 35.1169
[54]	training's rmse: 5.19525	training's l2: 26.9906	valid_1's rmse: 5.92281	valid_1's l2: 35.0796
[55]	training's rmse: 5.1829	training's l2: 26.8624	valid_1's rmse: 5.92673	valid_1's l2: 35.1261
[56]	training's 

[132]	training's rmse: 4.52148	training's l2: 20.4438	valid_1's rmse: 5.82632	valid_1's l2: 33.946
[133]	training's rmse: 4.51401	training's l2: 20.3763	valid_1's rmse: 5.82682	valid_1's l2: 33.9519
[134]	training's rmse: 4.50409	training's l2: 20.2869	valid_1's rmse: 5.82575	valid_1's l2: 33.9394
[135]	training's rmse: 4.49999	training's l2: 20.2499	valid_1's rmse: 5.82873	valid_1's l2: 33.9741
[136]	training's rmse: 4.49527	training's l2: 20.2075	valid_1's rmse: 5.82876	valid_1's l2: 33.9745
[137]	training's rmse: 4.48975	training's l2: 20.1579	valid_1's rmse: 5.82621	valid_1's l2: 33.9448
[138]	training's rmse: 4.48112	training's l2: 20.0804	valid_1's rmse: 5.82742	valid_1's l2: 33.9589
[139]	training's rmse: 4.47274	training's l2: 20.0054	valid_1's rmse: 5.82604	valid_1's l2: 33.9428
[140]	training's rmse: 4.46785	training's l2: 19.9617	valid_1's rmse: 5.82311	valid_1's l2: 33.9087
[141]	training's rmse: 4.4596	training's l2: 19.888	valid_1's rmse: 5.8251	valid_1's l2: 33.9318
[142

[23]	training's rmse: 5.61905	training's l2: 31.5737	valid_1's rmse: 5.98812	valid_1's l2: 35.8576
[24]	training's rmse: 5.60157	training's l2: 31.3776	valid_1's rmse: 5.98059	valid_1's l2: 35.7675
[25]	training's rmse: 5.58466	training's l2: 31.1885	valid_1's rmse: 5.97336	valid_1's l2: 35.6811
[26]	training's rmse: 5.56672	training's l2: 30.9884	valid_1's rmse: 5.96614	valid_1's l2: 35.5948
[27]	training's rmse: 5.5491	training's l2: 30.7925	valid_1's rmse: 5.95825	valid_1's l2: 35.5007
[28]	training's rmse: 5.53353	training's l2: 30.62	valid_1's rmse: 5.95492	valid_1's l2: 35.4611
[29]	training's rmse: 5.5174	training's l2: 30.4417	valid_1's rmse: 5.94641	valid_1's l2: 35.3598
[30]	training's rmse: 5.50261	training's l2: 30.2787	valid_1's rmse: 5.94488	valid_1's l2: 35.3417
[31]	training's rmse: 5.48677	training's l2: 30.1047	valid_1's rmse: 5.94182	valid_1's l2: 35.3052
[32]	training's rmse: 5.47296	training's l2: 29.9533	valid_1's rmse: 5.94364	valid_1's l2: 35.3268
[33]	training'

[107]	training's rmse: 4.74369	training's l2: 22.5026	valid_1's rmse: 5.80763	valid_1's l2: 33.7286
[108]	training's rmse: 4.73321	training's l2: 22.4033	valid_1's rmse: 5.80851	valid_1's l2: 33.7388
[109]	training's rmse: 4.72547	training's l2: 22.3301	valid_1's rmse: 5.80397	valid_1's l2: 33.686
[110]	training's rmse: 4.71873	training's l2: 22.2664	valid_1's rmse: 5.80187	valid_1's l2: 33.6617
[111]	training's rmse: 4.71132	training's l2: 22.1965	valid_1's rmse: 5.79812	valid_1's l2: 33.6182
[112]	training's rmse: 4.7023	training's l2: 22.1116	valid_1's rmse: 5.79849	valid_1's l2: 33.6225
[113]	training's rmse: 4.69471	training's l2: 22.0403	valid_1's rmse: 5.80057	valid_1's l2: 33.6466
[114]	training's rmse: 4.68108	training's l2: 21.9125	valid_1's rmse: 5.80338	valid_1's l2: 33.6792
[115]	training's rmse: 4.67187	training's l2: 21.8264	valid_1's rmse: 5.79871	valid_1's l2: 33.6251
[116]	training's rmse: 4.66623	training's l2: 21.7737	valid_1's rmse: 5.80029	valid_1's l2: 33.6433
[1

5.739586170043229
[5.5066332681705665, 5.903530660108021, 5.824446629361653, 5.972301155553186, 5.775035039443208, 5.942408269720643, 5.5597616417281115, 5.689604049548939, 5.981828586474166, 5.748478669390767, 5.753355752475107, 5.673323699778489, 5.954458674893091, 5.778924313395715, 5.739586170043229]


In [116]:
lgb_final = lgb_model.predict(test)

In [117]:
submit['target'] = lgb_final

In [118]:
submit.to_csv('klgb.csv', index=False)

In [119]:
svr_model = SVR()

In [120]:
svr_model.fit(X_train, y_train)

SVR()

In [121]:
svr_pred = svr_model.predict(X_test)

In [122]:
np.sqrt(mean_squared_error_squared_error_squared_error(y_test, svr_pred))

7.656324967828662

In [123]:
svr_final = svr_model.predict(test)

In [124]:
submit['target'] = svr_final

In [125]:
submit.to_csv('svr.csv', index=False)

In [126]:
lin_model = LinearRegression()

In [127]:
lin_model.fit(X_train, y_train)

LinearRegression()

In [128]:
lin_pred = lin_model.predict(X_test)

In [129]:
np.sqrt(mean_squared_error(y_test, lin_pred))

7.022736712400937

In [130]:
lin_final = lin_model.predict(test)

In [131]:
submit['target'] = lin_final

In [133]:
submit.to_csv('lin.csv', index=False)

In [134]:
paramet = {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 'min_samples_split': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]}

In [137]:
rand_search = RandomizedSearchCV(estimator=dc_model, param_distributions=paramet, cv=5, n_jobs=-1)

In [138]:
rand_search.fit(X, y)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10, 11,
                                                              12, 13, 14, 15]})

In [139]:
print('Result:')
print('\n the best estimator across all searched parmas: ', rand_search.best_estimator_)
print('\n the best score across all searched params: ', rand_search.best_score_)
print('\n the best parameters acress all best searched params: ', rand_search.best_params_)

Result:

 the best estimator across all searched parmas:  DecisionTreeRegressor(max_depth=5, min_samples_leaf=2, min_samples_split=3)

 the best score across all searched params:  0.20313940577249562

 the best parameters acress all best searched params:  {'min_samples_split': 3, 'min_samples_leaf': 2, 'max_depth': 5}


In [144]:
kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [145]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    dc_model.fit(X_train, y_train)
    scores.append(rmse(y_test, dc_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
5.954074685609762
============fold1=============
6.475799150338886
============fold2=============
6.303268953836365
============fold3=============
6.392960901710669
============fold4=============
6.235460310136136
============fold5=============
6.459092451490343
============fold6=============
6.1314512014122755
============fold7=============
6.250687100472684
============fold8=============
6.426304220014246
============fold9=============
6.216320894607902
============fold10=============
6.30531243147495
============fold11=============
6.246666655175143
============fold12=============
6.335981007124681
============fold13=============
6.35567159983991
============fold14=============
6.342776767786751
[5.954074685609762, 6.475799150338886, 6.303268953836365, 6.392960901710669, 6.235460310136136, 6.459092451490343, 6.1314512014122755, 6.250687100472684, 6.426304220014246, 6.216320894607902, 6.30531243147495, 6.246666655175143, 6.335981007124681, 6.35567159983

In [143]:
dc_model = DecisionTreeRegressor(min_samples_leaf=3, min_samples_split=2, max_depth=5)

In [146]:
dc_final = dc_model.predict(test)

In [147]:
submit['target'] = dc_final

In [148]:
submit.to_csv('kdc.csv', index=False)

def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor(n_neighbors=5)
    models['rfc'] = RandomForestRegressor(n_estimators = 210, min_samples_split = 18, max_depth = 26)
    models['xgb'] = XGBRegressor(learning_rate =0.05, n_estimators=135, max_depth=17, reg_lambda=12)
    
    return models

def evaluate_model(model):
    cv = RepeatedKFold(n_splits=15, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    
    return scores

def get_stacking():
    level0 = list()
    level0.append(('knn', KNeighborsRegressor(n_neighbors=5)))
    level0.append(('rfc', RandomForestRegressor(n_estimators = 210, min_samples_split = 18, max_depth = 26)))
    level0.append(('xgb', XGBRegressor(learning_rate =0.05, n_estimators=135, max_depth=17, reg_lambda=12)))
    
    level1 = cat_model = CatBoostRegressor(learning_rate=0.15, iterations=115, depth=15)
    
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    
    return model

def get_models_final():
    models = dict()
    models['knn'] = KNeighborsRegressor(n_neighbors=5)
    models['rfc'] = RandomForestRegressor(n_estimators = 210, min_samples_split = 18, max_depth = 26)
    models['xgb'] = XGBRegressor(learning_rate =0.05, n_estimators=135, max_depth=17, reg_lambda=12)
    models['stacking'] = get_stacking()
    
    return models

In [152]:
#final model for prediction

level0 = list()
level0.append(('knn', KNeighborsRegressor(n_neighbors=5)))
level0.append(('rfc', RandomForestRegressor(n_estimators = 210, min_samples_split = 18, max_depth = 26)))
level0.append(('xgb', XGBRegressor(learning_rate =0.05, n_estimators=135, max_depth=17, reg_lambda=12)))
level0.append(('lgb', LGBMRegressor(reg_lambda=14, num_leaves=50, n_estimators=195, max_depth=19, learning_rate=0.16)))
    
level1 = cat_model = CatBoostRegressor(learning_rate=0.15, iterations=115, depth=15)
    
final_model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5, passthrough=True, )

In [150]:
kf = KFold(n_splits=15, shuffle=True, random_state=30)

In [153]:
scores = []

for fold, (tr_in, te_in) in enumerate(kf.split(X)):
    print(f'============fold{fold}=============')
    X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
    y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
    final_model.fit(X_train, y_train)
    scores.append(rmse(y_test, final_model.predict(X_test)))
    print(scores[-1])
    
print(scores)

============fold0=============
0:	learn: 6.7845726	total: 6.78s	remaining: 12m 52s
1:	learn: 6.5354847	total: 13.6s	remaining: 12m 45s
2:	learn: 6.3313527	total: 20.2s	remaining: 12m 35s
3:	learn: 6.1764080	total: 26.9s	remaining: 12m 27s
4:	learn: 6.0483090	total: 33.6s	remaining: 12m 19s
5:	learn: 5.9536340	total: 40.3s	remaining: 12m 11s
6:	learn: 5.8656519	total: 47s	remaining: 12m 4s
7:	learn: 5.8001845	total: 53.7s	remaining: 11m 57s
8:	learn: 5.7296148	total: 1m	remaining: 11m 50s
9:	learn: 5.6484168	total: 1m 7s	remaining: 11m 43s
10:	learn: 5.5907833	total: 1m 13s	remaining: 11m 37s
11:	learn: 5.5528356	total: 1m 20s	remaining: 11m 30s
12:	learn: 5.5056322	total: 1m 27s	remaining: 11m 23s
13:	learn: 5.4223373	total: 1m 33s	remaining: 11m 16s
14:	learn: 5.3670816	total: 1m 40s	remaining: 11m 10s
15:	learn: 5.3316459	total: 1m 47s	remaining: 11m 7s
16:	learn: 5.2584883	total: 1m 54s	remaining: 11m 1s
17:	learn: 5.2326594	total: 2m 1s	remaining: 10m 54s
18:	learn: 5.1977840	total

39:	learn: 4.4302666	total: 4m 29s	remaining: 8m 24s
40:	learn: 4.4263822	total: 4m 35s	remaining: 8m 17s
41:	learn: 4.3929450	total: 4m 42s	remaining: 8m 11s
42:	learn: 4.3518396	total: 4m 49s	remaining: 8m 4s
43:	learn: 4.3370330	total: 4m 55s	remaining: 7m 57s
44:	learn: 4.3121964	total: 5m 2s	remaining: 7m 50s
45:	learn: 4.2714276	total: 5m 9s	remaining: 7m 43s
46:	learn: 4.2558362	total: 5m 15s	remaining: 7m 37s
47:	learn: 4.2393383	total: 5m 22s	remaining: 7m 30s
48:	learn: 4.2256091	total: 5m 29s	remaining: 7m 23s
49:	learn: 4.2039580	total: 5m 36s	remaining: 7m 16s
50:	learn: 4.1819878	total: 5m 42s	remaining: 7m 10s
51:	learn: 4.1725734	total: 5m 49s	remaining: 7m 3s
52:	learn: 4.1623444	total: 5m 56s	remaining: 6m 56s
53:	learn: 4.1331443	total: 6m 3s	remaining: 6m 50s
54:	learn: 4.1080840	total: 6m 9s	remaining: 6m 43s
55:	learn: 4.0900884	total: 6m 16s	remaining: 6m 36s
56:	learn: 4.0680038	total: 6m 23s	remaining: 6m 29s
57:	learn: 4.0401096	total: 6m 29s	remaining: 6m 23s

79:	learn: 3.6747697	total: 8m 54s	remaining: 3m 53s
80:	learn: 3.6604425	total: 9m	remaining: 3m 47s
81:	learn: 3.6557085	total: 9m 7s	remaining: 3m 40s
82:	learn: 3.6310944	total: 9m 14s	remaining: 3m 33s
83:	learn: 3.6304582	total: 9m 20s	remaining: 3m 26s
84:	learn: 3.5864142	total: 9m 27s	remaining: 3m 20s
85:	learn: 3.5824125	total: 9m 34s	remaining: 3m 13s
86:	learn: 3.5806038	total: 9m 41s	remaining: 3m 7s
87:	learn: 3.5610857	total: 9m 47s	remaining: 3m
88:	learn: 3.5501898	total: 9m 54s	remaining: 2m 53s
89:	learn: 3.4991606	total: 10m 1s	remaining: 2m 47s
90:	learn: 3.4669695	total: 10m 8s	remaining: 2m 40s
91:	learn: 3.4566227	total: 10m 14s	remaining: 2m 33s
92:	learn: 3.4310487	total: 10m 21s	remaining: 2m 27s
93:	learn: 3.4191638	total: 10m 28s	remaining: 2m 20s
94:	learn: 3.3920305	total: 10m 34s	remaining: 2m 13s
95:	learn: 3.3891628	total: 10m 41s	remaining: 2m 6s
96:	learn: 3.3876094	total: 10m 48s	remaining: 2m
97:	learn: 3.3738006	total: 10m 54s	remaining: 1m 53s
9

3:	learn: 6.1676188	total: 26.7s	remaining: 12m 19s
4:	learn: 6.0414139	total: 33.3s	remaining: 12m 12s
5:	learn: 5.9404766	total: 40s	remaining: 12m 6s
6:	learn: 5.8473171	total: 46.6s	remaining: 11m 59s
7:	learn: 5.7802995	total: 53.3s	remaining: 11m 52s
8:	learn: 5.7085814	total: 60s	remaining: 11m 46s
9:	learn: 5.6210499	total: 1m 6s	remaining: 11m 39s
10:	learn: 5.5724783	total: 1m 13s	remaining: 11m 32s
11:	learn: 5.5298357	total: 1m 19s	remaining: 11m 25s
12:	learn: 5.4873725	total: 1m 26s	remaining: 11m 19s
13:	learn: 5.3979058	total: 1m 33s	remaining: 11m 12s
14:	learn: 5.3429083	total: 1m 40s	remaining: 11m 7s
15:	learn: 5.3042383	total: 1m 46s	remaining: 11m 1s
16:	learn: 5.2535553	total: 1m 53s	remaining: 10m 54s
17:	learn: 5.2261374	total: 2m	remaining: 10m 47s
18:	learn: 5.1846693	total: 2m 6s	remaining: 10m 40s
19:	learn: 5.1330103	total: 2m 13s	remaining: 10m 34s
20:	learn: 5.1035419	total: 2m 20s	remaining: 10m 27s
21:	learn: 5.0507345	total: 2m 26s	remaining: 10m 20s


43:	learn: 4.4091442	total: 4m 49s	remaining: 7m 47s
44:	learn: 4.3733607	total: 4m 56s	remaining: 7m 40s
45:	learn: 4.3478778	total: 5m 2s	remaining: 7m 34s
46:	learn: 4.3205311	total: 5m 9s	remaining: 7m 27s
47:	learn: 4.2887705	total: 5m 16s	remaining: 7m 21s
48:	learn: 4.2579188	total: 5m 22s	remaining: 7m 14s
49:	learn: 4.2283814	total: 5m 29s	remaining: 7m 8s
50:	learn: 4.2057690	total: 5m 36s	remaining: 7m 1s
51:	learn: 4.2055012	total: 5m 36s	remaining: 6m 47s
52:	learn: 4.1958948	total: 5m 42s	remaining: 6m 41s
53:	learn: 4.1872456	total: 5m 49s	remaining: 6m 35s
54:	learn: 4.1445742	total: 5m 56s	remaining: 6m 28s
55:	learn: 4.1311811	total: 6m 3s	remaining: 6m 22s
56:	learn: 4.1165958	total: 6m 9s	remaining: 6m 16s
57:	learn: 4.0950351	total: 6m 16s	remaining: 6m 10s
58:	learn: 4.0653755	total: 6m 23s	remaining: 6m 3s
59:	learn: 4.0544270	total: 6m 29s	remaining: 5m 57s
60:	learn: 4.0287638	total: 6m 36s	remaining: 5m 51s
61:	learn: 3.9840466	total: 6m 43s	remaining: 5m 44s


...

47:	learn: 4.3427296	total: 5m 20s	remaining: 7m 27s
48:	learn: 4.3335188	total: 5m 27s	remaining: 7m 20s
49:	learn: 4.2989378	total: 5m 33s	remaining: 7m 14s
50:	learn: 4.2828091	total: 5m 40s	remaining: 7m 7s
51:	learn: 4.2678693	total: 5m 47s	remaining: 7m
52:	learn: 4.2480126	total: 5m 53s	remaining: 6m 53s
53:	learn: 4.2340811	total: 6m	remaining: 6m 47s
54:	learn: 4.2015404	total: 6m 7s	remaining: 6m 40s
55:	learn: 4.1826928	total: 6m 13s	remaining: 6m 33s
56:	learn: 4.1699973	total: 6m 20s	remaining: 6m 27s
57:	learn: 4.1409944	total: 6m 27s	remaining: 6m 20s
58:	learn: 4.1242534	total: 6m 33s	remaining: 6m 13s
59:	learn: 4.1141839	total: 6m 40s	remaining: 6m 6s
60:	learn: 4.0991713	total: 6m 47s	remaining: 6m
61:	learn: 4.0885437	total: 6m 53s	remaining: 5m 53s
62:	learn: 4.0600341	total: 7m	remaining: 5m 47s
63:	learn: 4.0464191	total: 7m 7s	remaining: 5m 40s
64:	learn: 4.0344149	total: 7m 13s	remaining: 5m 33s
65:	learn: 3.9954335	total: 7m 20s	remaining: 5m 26s
66:	learn: 3.

87:	learn: 3.4872900	total: 9m 43s	remaining: 2m 59s
88:	learn: 3.4652661	total: 9m 50s	remaining: 2m 52s
89:	learn: 3.4616752	total: 9m 57s	remaining: 2m 45s
90:	learn: 3.4376785	total: 10m 3s	remaining: 2m 39s
91:	learn: 3.4000706	total: 10m 10s	remaining: 2m 32s
92:	learn: 3.3837441	total: 10m 17s	remaining: 2m 26s
93:	learn: 3.3479390	total: 10m 23s	remaining: 2m 19s
94:	learn: 3.3440729	total: 10m 30s	remaining: 2m 12s
95:	learn: 3.3268528	total: 10m 37s	remaining: 2m 6s
96:	learn: 3.3256413	total: 10m 43s	remaining: 1m 59s
97:	learn: 3.2960570	total: 10m 50s	remaining: 1m 52s
98:	learn: 3.2880924	total: 10m 57s	remaining: 1m 46s
99:	learn: 3.2793473	total: 11m 4s	remaining: 1m 39s
100:	learn: 3.2516877	total: 11m 10s	remaining: 1m 33s
101:	learn: 3.2354924	total: 11m 17s	remaining: 1m 26s
102:	learn: 3.2295752	total: 11m 24s	remaining: 1m 19s
103:	learn: 3.1986955	total: 11m 30s	remaining: 1m 13s
104:	learn: 3.1778125	total: 11m 37s	remaining: 1m 6s
105:	learn: 3.1539432	total: 1

10:	learn: 5.5496543	total: 1m 13s	remaining: 11m 32s
11:	learn: 5.5054471	total: 1m 19s	remaining: 11m 25s
12:	learn: 5.4677407	total: 1m 26s	remaining: 11m 19s
13:	learn: 5.4267267	total: 1m 33s	remaining: 11m 12s
14:	learn: 5.3539675	total: 1m 39s	remaining: 11m 5s
15:	learn: 5.3061489	total: 1m 46s	remaining: 10m 58s
16:	learn: 5.2745701	total: 1m 53s	remaining: 10m 52s
17:	learn: 5.2387075	total: 1m 59s	remaining: 10m 45s
18:	learn: 5.1867462	total: 2m 6s	remaining: 10m 38s
19:	learn: 5.1171586	total: 2m 13s	remaining: 10m 32s
20:	learn: 5.0836677	total: 2m 19s	remaining: 10m 25s
21:	learn: 5.0502644	total: 2m 26s	remaining: 10m 18s
22:	learn: 5.0152933	total: 2m 32s	remaining: 10m 11s
23:	learn: 4.9895458	total: 2m 39s	remaining: 10m 5s
24:	learn: 4.9620704	total: 2m 46s	remaining: 9m 58s
25:	learn: 4.9343272	total: 2m 53s	remaining: 9m 52s
26:	learn: 4.8783835	total: 2m 59s	remaining: 9m 46s
27:	learn: 4.8694189	total: 3m 6s	remaining: 9m 39s
28:	learn: 4.8423118	total: 3m 13s	r

49:	learn: 4.3346633	total: 5m 21s	remaining: 6m 58s
50:	learn: 4.3136952	total: 5m 28s	remaining: 6m 52s
51:	learn: 4.3129707	total: 5m 28s	remaining: 6m 38s
52:	learn: 4.2856472	total: 5m 35s	remaining: 6m 32s
53:	learn: 4.2585763	total: 5m 41s	remaining: 6m 26s
54:	learn: 4.2264583	total: 5m 48s	remaining: 6m 20s
55:	learn: 4.2123077	total: 5m 55s	remaining: 6m 14s
56:	learn: 4.1678358	total: 6m 1s	remaining: 6m 8s
57:	learn: 4.1513801	total: 6m 8s	remaining: 6m 2s
58:	learn: 4.1311259	total: 6m 15s	remaining: 5m 56s
59:	learn: 4.1247155	total: 6m 21s	remaining: 5m 49s
60:	learn: 4.1072056	total: 6m 28s	remaining: 5m 43s
61:	learn: 4.0899763	total: 6m 35s	remaining: 5m 37s
62:	learn: 4.0627620	total: 6m 41s	remaining: 5m 31s
63:	learn: 4.0405454	total: 6m 48s	remaining: 5m 25s
64:	learn: 4.0307561	total: 6m 55s	remaining: 5m 19s
65:	learn: 4.0059627	total: 7m 1s	remaining: 5m 13s
66:	learn: 3.9845925	total: 7m 8s	remaining: 5m 7s
67:	learn: 3.9427770	total: 7m 15s	remaining: 5m
68:	

89:	learn: 3.6295441	total: 9m 31s	remaining: 2m 38s
90:	learn: 3.5933802	total: 9m 38s	remaining: 2m 32s
91:	learn: 3.5849812	total: 9m 45s	remaining: 2m 26s
92:	learn: 3.5577008	total: 9m 51s	remaining: 2m 19s
93:	learn: 3.5370658	total: 9m 58s	remaining: 2m 13s
94:	learn: 3.5084145	total: 10m 5s	remaining: 2m 7s
95:	learn: 3.4988187	total: 10m 11s	remaining: 2m 1s
96:	learn: 3.4780379	total: 10m 18s	remaining: 1m 54s
97:	learn: 3.4481303	total: 10m 25s	remaining: 1m 48s
98:	learn: 3.4162570	total: 10m 31s	remaining: 1m 42s
99:	learn: 3.3972178	total: 10m 38s	remaining: 1m 35s
100:	learn: 3.3829993	total: 10m 45s	remaining: 1m 29s
101:	learn: 3.3674042	total: 10m 51s	remaining: 1m 23s
102:	learn: 3.3461670	total: 10m 58s	remaining: 1m 16s
103:	learn: 3.3311823	total: 11m 5s	remaining: 1m 10s
104:	learn: 3.3032832	total: 11m 12s	remaining: 1m 4s
105:	learn: 3.3023700	total: 11m 18s	remaining: 57.6s
106:	learn: 3.2819143	total: 11m 25s	remaining: 51.3s
107:	learn: 3.2554467	total: 11m 

In [154]:
final_final = final_model.predict(test)

In [155]:
submit['target'] = final_final

In [156]:
submit.to_csv('stacking.csv', index=False)